#Imágenes en Google Earth Engine

---



Los datos raster en Earth Engine se representan mediante el objeto **Image**. 

Como ya se sabe, una imagen se compone de una o varias bandas y cada banda tiene su propio nombre, tipo de dato, escala... Además cada imagen tiene unos metadatos almacenados en un diccionario. A lo largo de los sucesivos cuadernos se estudiará como  acceder a una o varias imágenes de satélite a través del uso de estos metadatos.

Concretamente en este cuaderno se van a estudiar los siguientes puntos:
1. Introducción a Image.
2. Metadatos e información de imágenes
3. Visualización de imágenes
4. Expresiones matemáticas

Además de estas funcionalidades es posible aplicar métodos de convolución, operadore morfológicos, gradientes, texturas, etc...

## **2.0 Preparación del entorno**

Como ya se ha visto anteriormente, lo primero que se debe hacer es preparar el entorno de desarrollo.

In [2]:
import ee

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=mrDUxYj3UM87Tl_TXr3cWXDdR46Gkm1P7pnmbV_hHg8&tc=vhVQcP02YLUfdN6JQH1TyFpmenD-pudWFNaP-VO1jc0&cc=UMbExuC2FYXIL6vNvfLE2zWwylWmAkfZRK7r7sRzJZ8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk4jXBzN9NhG-7SVpd8YpIF3XiK5aZHumpdXg7mF9WBiUNPXFPxaoHQ

Successfully saved authorization token.


## **2.1 Introducción a Image**

Es posible a acceder a una imagen particular dentro del repositorio del servicio GEE si se conoce su identificador en los servidores de Google. Además, tambien es posible crear una imagen a partir de una constante, listas o cualquier otro objeto adecuado de GEE.

En primer lugar se va a aprender como crear una imagen y manipular sus bandas.

### **2.1.1 Creación de una imagen constante**
En este script se ha creado una imagen sintética almacenada en la variable `imagen1` la cual contiene un pixel con valor igual a 1. 
A continuación se imprime en la consola el contenido de dicha variable, mostrándonos el contenido asociada a la variable.

Posteriormente se imprime la información asociada a la variable a través de **getInfo()**, monstrando información de sus metadatos. En este caso se esta indicando que la imagen tiene asociadao como sistema de referencia WGS84, el cual se corresponde con el código EPGS 4326. Igualmente se indica cuales son los valores máximos y mínimos en la imagen así como el tipo de dato almacenado en los póxeles, en este caso de tipo entero.

Cuando se acceda a una imagen de satélite se podrá ver muchas mas información asociada a la misma.

Para mas información sobre códigos EPSG (*European Petroleum Survey Group*) consultar la web https://spatialreference.org/.


In [3]:
imagen1=ee.Image(100) #Creación de una varialbe de tipo imagen
print (imagen1) 
imagen1.getInfo() #Muestra la información de la imagen

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.constant",
    "arguments": {
      "value": {
        "constantValue": 100
      }
    }
  }
})


{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 100,
    'max': 100},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

En el siguiente ejemplo se crea una imagen sintética que tiene 3 bandas espectrales de un pixel en cada una de ellas, siendo los valores de cada uno 1,-1 y 120.

In [ ]:
#Creación de una imagen multibanda a partir de una lista de constantes
imagen=ee.Image([1,-1,120])
imagen.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 1, 'min': 1, 'precision': 'int', 'type': 'PixelType'},
   'id': 'constant'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': -1,
    'min': -1,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'constant_1'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 120,
    'min': 120,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'constant_2'}],
 'type': 'Image'}


### **2.1.2 Concatenar imágenes**

En algunas ocasiones puede ser necesario juntar o concatenar distintas imágenes las cuales darán lugar a una única imagen. 

Esta operación se realizará mediante **ee.Image.cat(*list*)**, este método concatena un conjunto de imágenes que se pasan a través de una lista en una sola imagen.

In [4]:
#Concatenación de tres imágenes en una imagen individual de tres bandas
imagen1=ee.Image(1)
imagen2=ee.Image(120)
imagen3=ee.Image(-1)

imagen=ee.Image.cat([imagen1,imagen2,imagen3]) #Concatena las tres imágenes individuales en una sola
imagen.getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'constant_1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 120,
    'max': 120},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'constant_2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -1,
    'max': -1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

Un método muy interesante asociado a una variable de tipo ee.Image es **addBands()**. Este método añade las bandas pasadas como parámetros a las ya existentes en una imagen ya previamente creada.

In [5]:
#En este ejemplo se añade una nueva banda a la imagen multibanda creada anteriormente
imagen=imagen.addBands(ee.Image(10))
imagen.getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'constant_1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 120,
    'max': 120},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'constant_2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -1,
    'max': -1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'constant_3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 10,
    'max': 10},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

## **2. Información de las imágenes: Metadatos**

Como ya se ha visto anteriormente, para conocer la información asociada a las imagenes y/o de las bandas espectrales de éstas se emplea el método **getInfo()**. 

En los metadatos se encuentra toda la información relativa a la fecha de visita del satélite, las bandas espectrales que presenta, el porcentaje de nubes o las coordenadas de la huella proyectada sobre el terreno entre otros datos. Esta información se empleará por ejemplo para crear *colecciones de imágenes* basada en búsquedas en un repositoro.

En el siguiente ejemplo se va a obtener información de todos los metadatos de la imagen, grabándola en la varialbe *info*, posteriormente se imprime esta en pantalla.


In [6]:
#Cargamos una imagen
imagen=ee.Image('COPERNICUS/S2/20170412T110621_20170412T111708_T30SUG')

info=imagen.getInfo()

print (imagen.getInfo())


{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32630', 'crs_transform': [60, 0, 300000, 0, -60, 4200000]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32630', 'crs_transform': [10, 0, 300000, 0, -10, 4200000]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32630', 'crs_transform': [10, 0, 300000, 0, -10, 4200000]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32630', 'crs_transform': [10, 0, 300000, 0, -10, 4200000]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32630', 'crs_transform': [20, 0, 300000, 0, -20, 4200000

Si directamente se imprime en pantalla, sin almacenarla en una variable, se verá la información de cada elemento una de los elementos de los metadatos y su valor.

In [7]:
imagen.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32630',
   'crs_transform': [60, 0, 300000, 0, -60, 4200000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32630',
   'crs_transform': [10, 0, 300000, 0, -10, 4200000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32630',
   'crs_transform': [10, 0, 300000, 0, -10, 4200000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32630',
   'crs_transform': [10, 0, 300000, 0, -10, 4200000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min':

GEE ofrece numerosos dataset asociados con imágenes, bien sea directamente imágenes de satélite, datos de clima, modelos digitales de elevaciones, etc. Para mas información de todos los dataset que ofrece GEE consultar este [link](https://developers.google.com/earth-engine/datasets/).

En lo que respecta a escenas de satélite el usuario puede encontrar toda la colección de imágenes de los programas de observación de la Tierra [Copernicus](https://developers.google.com/earth-engine/datasets/catalog/sentinel), [Landsat](https://developers.google.com/earth-engine/datasets/catalog/landsat) o [Modis](https://developers.google.com/earth-engine/datasets/catalog/modis).Dentro de cada unos de estos programas el usario puede acceder a  distintos productos como imágenes corregidas atmosféricamente, datos brutos, imágenes sintéticas, etc...De este modo, dentro de cada programa de observación de la Tierra y tipo de producto aparece un dataset el cual tiene asociado un identificador que aparece en el apartado **Earth Engine Snippet**. Por ejemplo, en el caso de las imágenes de Sentinel 5 con información de dióxido de nitrogeno en atmósfera se puede ver como éstas estan disponibles desde el 2018-06-28, que el proveedor de los datos es la ESA y que los datos se encuentran en `COPERNICUS/S5P/OFFL/L3_N02`.


![](https://www.gisandbeers.com/wp-content/uploads/2019/04/Colecciones-de-imágenes-en-Google-Earth-Engine.jpg)

Por tanto, se debe navegar por los distintos datasetes de GEE, encontrar el que sea de interes del usuario/proyecto y emplear su identificador para acceder a su contenido. En segunto lugar, se deberá de saber el nombre la imagen o identificor que se quiere emplear. 

En el siguiente código se verá como poder localizar las imágenes que interesen para una determinada aplicación para abordar un trabajo en concreto, por el momento se dará por hecho que se conoce el nombre o identificador de la escena a emplear.

Por ejemplo, en el código anterior se accede a una imagen Sentinel 2 con valores de reflectancia a nivel de atmósfera como se puede verificar en la web del [producto](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2):

`imagen=ee.Image('COPERNICUS/S2/20170412T110621_20170412T111708_T30SUG')`

Además, para cada dataset aparece una descripción de las bandas y otras propiedadades de las escenas.

Se presentan ahora distintos ejemplos solicitando información de algunos elementos de metadatos concretos para los que GEE tiene implementados métodos concretos.

*   **bandNames()**: Devuelve una lista con el nombre de las imágenes.
*   **projection()**: Informa del Sistema de Referencia de Coordenadas de la imagen.
*   **select()**: Selecciona una banda espectral de las presentes en la escena.
*   **projection().nominalScale()**: Obtiene el valor del tamaño del pixel de la banda.
*   **propertyNames()**: Lista el nombre de los elementos de los metadatos de la imagen.
*   **get()**: Extrae un elemento concreto de los metadatos


In [8]:
imagen=ee.Image('COPERNICUS/S2/20170412T110621_20170412T111708_T30SUG')

nombre_bandas=imagen.bandNames() #Se obtiene una lista con el nombre de las bandas de una imagen.

banda4= imagen.select('B4') #Creamos una varia de tipo imagen que es igual a la banda 4 de la variable imagen
banda10=imagen.select('B10') #Igual pero seleccionando la banda 10 de la variable imagen
SRC = banda4.projection() #Imprime en pantalla el sistema de referencia de coordenadas asociado a la imagen

pixel_size4= banda4.projection().nominalScale() #Almacena el tamaño de pixel de la banda 4
pixel_size10= banda10.projection().nominalScale() #Almacena el tamaño de pixel de la banda 10

metadatos_nombres= imagen.propertyNames() #Obtenemos la lista de los nombres de los metadatos

nubes=imagen.get('CLOUDY_PIXEL_PERCENTAGE') #Obtenemos el porcentaje de nubes en la imagen
huella=imagen.get('system:footprint') #Información de la huella de la imagen sobre el terreno


print ("El nombre de las bandas es:")
print (nombre_bandas.getInfo(),"\n")
print ("El SRC de la banda 4 es:")
print (SRC.getInfo(),"\n")

print ("Tamaño pixel banda 4: ", pixel_size4.getInfo(),"\n")
print ("Tamaño pixel banda 10: ", pixel_size10.getInfo(),"\n")

print ("Nombre de los metadatos:","\n")
print (metadatos_nombres.getInfo(),"\n")

print ("Porcentaje de nubes: ", nubes.getInfo())
print ("Huella de la escena: ",huella.getInfo())

El nombre de las bandas es:
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60'] 

El SRC de la banda 4 es:
{'type': 'Projection', 'crs': 'EPSG:32630', 'transform': [10, 0, 300000, 0, -10, 4200000]} 

Tamaño pixel banda 4:  10 

Tamaño pixel banda 10:  60 

Nombre de los metadatos: 

['DATATAKE_IDENTIFIER', 'SPACECRAFT_NAME', 'FORMAT_CORRECTNESS_FLAG', 'IERS_BULLETIN_FILENAME', 'system:id', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A', 'MEAN_SOLAR_AZIMUTH_ANGLE', 'system:footprint', 'SOLAR_IRRADIANCE_B12', 'system:version', 'SOLAR_IRRADIANCE_B10', 'SENSOR_QUALITY', 'SOLAR_IRRADIANCE_B11', 'GENERATION_TIME', 'SOLAR_IRRADIANCE_B8A', 'FORMAT_CORRECTNESS', 'PRODUCT_URI', 'SENSOR_QUALITY_FLAG', 'CLOUD_COVERAGE_ASSESSMENT', 'system:time_end', 'system:time_start', 'DATASTRIP_ID', 'PROCESSING_BASELINE', 'SENSING_ORBIT_NUMBER', 'GEOMETRIC_QUALITY_FLAG', 'SENSING_ORBIT_DIRECTION', 'GENERAL_QUALITY', 'GRANULE_ID', 'REFLECTANCE_CONVERSION_CORRECTION', 'M

En el siguiente ejemplo se presenta el código de una función en Python denominada **centro** que calcula las coordenadas del centro de la escena a partir de la información almacenada en el nodo de metadatos `system:footprint` de una escena Sentinel.

La función `centro` tendrá como entrada una variable de tipo image que se ha denominada imagen.

Para obtener las coordenadas de las vértices que describen la huella de la escena se consultará el elemento de metadatos **system:footprint**. Éste elemento contiene un subelemento denominado **coordinates** donde aparecen las coordenada de cada vértice que describe la huella de la escena proyectada sobre el terreno. 

Lo que hace la función es calcular el sumatorio de los valores de latitud y longitud recorriendo la lista mediante un bucle *for*. Una vez que termina este proceso se divide por el número de elementos, obteniendo la media de la longitud y la latitud, siendo devueltos estos valores por la función.



In [9]:
def centro(imagen):
  #Función para calcular las coordenadas del centro de una imagen
  coordenadas=imagen.get('system:footprint').getInfo()['coordinates']
  longitud=0
  latitud=0
  for i in range(len(coordenadas)):
    longitud= longitud+ coordenadas[i-1][0]
    latitud=latitud+coordenadas[i-1][1]
  
  longitud=longitud/(len(coordenadas))
  latitud=latitud/(len(coordenadas))
  return (longitud,latitud)

coordenadas_centrales=centro(imagen) #Llamada a la función centro

print ('longitud central: ',coordenadas_centrales[0])
print ('latitud central: ',coordenadas_centrales[1])


longitud central:  -4.703201349403696
latitud central:  37.468329527718225


## **3. Visualización de imagenes**

La API de Python no soporta la visualización interactiva de imágenes como lo hace el Code Editor de Google Earth Engine desarrollando en JavaScript.

Para la visualización de imágenes, ya se vio anteriormente que es posible hacer presentaciones estáticas o bien dinámicas a través del uso de un visor web. Para ello se empleará **ee.Image.getThumbURL**.

Al igual que en el caso de emplear Code Editor de GEE, se tiene acceso para controlar los siguientes parámetros:

<h2 style="color: #2e6c80;">&nbsp;</h2>
<table>
<tbody>
<tr>
<td style="text-align: center;"><strong>Par&aacute;metro</strong></td>
<td style="text-align: center;"><strong>Descripci&oacute;n</strong></td>
<td style="text-align: center;"><strong>Tipo</strong></td>
</tr>
<tr>
<td>&nbsp;bands</td>
<td>&nbsp;lista delimitada por comas de las bandas a representar en RGB</td>
<td>list&nbsp;</td>
</tr>
<tr>
<td>min&nbsp;</td>
<td>Valor(es) a representar como 0 o lista de tres n&uacute;meros, uno por cada banda.</td>
<td>&nbsp;n&uacute;mero o lista de tres n&uacute;meros</td>
</tr>
<tr>
<td>max&nbsp;</td>
<td>Valor(es) a representar como 255 o lista de tres n&uacute;meros, uno por cada banda.&nbsp;</td>
<td>n&uacute;mero o lista de tres n&uacute;meros&nbsp;</td>
</tr>
<tr>
<td>gain</td>
<td>Valor(es) con los que multiplicar cada valor de pixel a representar</td>
<td>n&uacute;mero o lista de tres n&uacute;meros&nbsp;</td>
</tr>
<tr>
<td>bias</td>
<td>Valor(es) que a&ntilde;adir a cada valor de pixel a representar</td>
<td>n&uacute;mero o lista de tres n&uacute;meros&nbsp;</td>
</tr>
<tr>
<td>gamma</td>
<td>Factor(es) de correcci&oacute;n gamma</td>
<td>n&uacute;mero o lista de tres n&uacute;meros&nbsp;</td>
</tr>
<tr>
<td>palette</td>
<td>Lista de estilos CSS con las cadenas de texto de color (solo para im&aacute;genes de una banda)</td>
<td>Lista separada por comas de cadenas hexadecimales</td>
</tr>
<tr>
<td>opacity</td>
<td>La opacidad de la capa (0.0 is completamente transparente y 1.0 completamente opaco)</td>
<td>N&uacute;mero</td>
</tr>
<tr>
<td>format</td>
<td>JPG o PNG</td>
<td>cadena</td>
</tr>
</tbody>
</table>
<p><strong>&nbsp;</strong></p>

Para la visualización dinámica de imágenes será necesario emplear paquetes / librerías externas como **folium**. Este paquete es el resultado de la unión del lenguaje de programación Python y **Leaflet**, permitiendo la manipulación de datos mediante el primero y su visualización por el segundo. En este sentido, Leaflet es una librería desarrollada en JavaScript que permite publicar mapas en entornos web ampliamente empleada hoy día.

En el siguiente código se facilita una función para incorporar capas de GEE en un mapa hecho en folium. Los parámetros a pasar a tal función serán: el objeto imagen, los parámetros de visualización y el nombre de la capa que se desea que aparezca en el control de capas.



In [10]:
# Import libreria Folium
import folium

# Definicón de una función para representar imágenes Earth Engine en un folium map.
# 
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Añade un método de dibujo en folium.
folium.Map.add_ee_layer = add_ee_layer

Una vez desarrollada la función para representar una imagen, en el siguiente ejemplo se va a representar una imagen Sentinel 2 de la cual se conoce su identificador.

En este ejemplo se empleará la función **add_ee_layer** para pintar la imagen en pantalla. 
Además como la escena de satélite puede estar en cualquier localización, se deberá centrar la vista del mapa, para ello se empleará la función creada anteriormente para determinar las coordenadas centrales de la imagen.

Finalmente, se han definido dos parámetros de visualización diferentes:
* vis_params_falso_color: empleado para representar la imagen Sentinel 2 con las bandas 8,3 y 4 correspondientes a las bandas Nir, Green and Red.
* vis_params_RGB: empleado para represental la imagen Sentinel 2 en color verdadero.


In [11]:
#Carga una imagen Sentinel 2
imagen=ee.Image('COPERNICUS/S2/20170412T110621_20170412T111708_T30SUG')

#Determinamox como se desea representar la imagen en pantalla, indicando las bandas a emplear, rango, etc.
#En este caso queremos representar las bandas B8, B3 y B4, empleando como rango dinámico 0 - 3500.
vis_params_falso_color = {
  'min': 0,
  'max': 3500,
  'bands': ['B8', 'B3', 'B4']
}
#Parámetros de representación en color verdadero
vis_params_RGB = {
  'min': 0,
  'max': 3500,
  'bands': ['B4', 'B3', 'B2']
}

#Llamada a la función centro para obtener las coordenada centrales de la escena
coordenadas_centrales=centro(imagen)

longitud=coordenadas_centrales[0]
latitud=coordenadas_centrales[1]
                 
# Creacion de un objeto de tipo folium map
# zoom_start indica el nivel de zoom en pantalla
# height indica la altura del visor que vamoa a crear en pantalla
mi_mapa = folium.Map(location=[latitud,longitud], zoom_start=12, height=1000)

# Añade la imagen al objeto mapa.
mi_mapa.add_ee_layer(imagen,vis_params_falso_color, 'Sentinel falso color') #Añade la imagen Sentinel considerando la representación en falso color
mi_mapa.add_ee_layer(imagen,vis_params_RGB, 'Sentinel RGB') #Añade la imagen Sentinel en modo RGB

# Añade un panel de control de capas en el mapa (se corresponde con el icono superior de la derecha del visor)
mi_mapa.add_child(folium.LayerControl())

# Muestra el mapa.
display(mi_mapa)

## **4. Operaciones sobre imágenes**

Google Earth Engine opera con las imágenes a nivel de pixel. Cuando un operador se aplica a una imagen, este se aplicará a todos los pixeles de la banda de una imagen. Pero, en el caso de que una imagen esté enmascarada, solo se aplicará a los píxeles dentro de la máscara.

Si se trabaja en la expresión con varias imágenes, la operación solo se aplicará en aquellos pixeles que estan dentro de la máscara en las dos imágenes.

Así, sobre una o varias bandas espectrales de una o varias escenas de satélite se pueden realizar operaciones de tipo:
* Algebraicas: sumar, restar, dividir, multiplicar.
* Trigonométricas: seno, coseno, tangente, arcoseno, arcocoseno, arcotangente.
* Booleanas, relacionales.
* Conversión de tipo.
* Otras como diferencia normalizada aplicación de máscara entre otras.


### **4.1 Operaciones algebraicas**

Este tipo de operaciones se pueden abordar bien mediante el uso de operadores matemáticos o bien mediante expresiones. Además, GEE presenta un método para calcular diferencias normalizadas entre los datos de bandas espectrales, muy útil en el caso del cálculo de índices como el índice de vegetación de diferencia normalizada (NDVI).

* **Operadores matemáticos:** Permiten realizar operaciones sencillas sobre los píxeles de una(s) banda(s) de una imagen(es). Ejemplo de estos operadores son **substract()**, **add()**, **divide()**, **multiply()** o **pow()**.

* **Expresiones**: El uso de operadores matemáticos puede resultar complejo, incluso para expresiones sencillas. Una alternativa es emplear el método **expresion()**. El primer argumento se corresponde con la representación en modo texto de la operación matemática a realizar. El segundo argumento es un diccionario donde las entradas son exactamente el mismo nombre de las variables empleada en la cadena de texto de la expresión, siendo sus valores las bandas a emplear. 

* **Diferencias normalizadas**: Un método muy util para calcular diferencias normalizadas entre bandas de imágenes tales como NDVI, NDWI o NBR es el médodo **normalizedDifference()**, pasando como parámetros dos bandas espectrales.

#### **Ejemplo de operadores matemáticos**


En este primer ejemplo se va a aprender como crear una imagen que contenga el resultado de calcular el índice de vegetación NDVI de una escena Sentinel 2 mediante operadores matemáticos.

En primer lugar, se muestra una figura con la curva espectral de una vegetación sana, con algun tipo de afección y suelo, permitiendo una mejor interpretación del índice a calcular.

![](http://physicsopenlab.org/wp-content/uploads/2017/01/veg.gif)

Fuente: http://physicsopenlab.org/2017/01/30/ndvi-index/.

Antes de calcular el índice es necesario conocer que región del espectro electromagnético cubren las distintas bandas que componen la imagen de trabajo. En este caso, para Sentinel 2 la descripción de las bandas espectrales se pueden consultar pulsando [aqui](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR).

Así,para escenas Sentinel 2 el NDVI se calculará como: $NDVI=\frac{NIR-Red}{NIR+Red}=\frac{B8-B4}{B8+B4}$

En el código se puede comprobar como se van encadenando distintos métodos:
*   *select*: selecciona la banda 8.
*   *substract*: calcula la diferencia entre la banda 8 y la 4 (tambien seleccionada).
*   *divide*: calcula el cociente entre la diferencia anterior y la suma de las dos bandas.
*   add: es el método para sumar la banda 8 y 4.

In [12]:
#Opcion 1: Calculo de NDVI mediante operadores matemáticos
#---------------------------------------------------------
#Carga una imagen Sentinel 2
imagen=ee.Image('COPERNICUS/S2/20170412T110621_20170412T111708_T30SUG')

# Seleccionamos la banda 4 y 8 correspondientes al rojo y al NIR
ndvi_opcion1 = imagen.select('B8').subtract(imagen.select('B4')).divide(imagen.select('B8').add(imagen.select('B4')))

#### **Ejemplo de expresion**


Al usar `expresion()` el primer argumento se corresponde con la representación en modo texto de la operación matemática. El segundo argumento es un diccionario donde las entradas son los nombres de las variables empleadas en la expresión, siendo sus valores las bandas a emplear. 

Las bandas espectrales de la imagen se referenciarán bien mediante b("nombre de bandas") o b(indice)

In [13]:
#Opción 2: Calculo de NDVI mediante expresiones
#----------------------------------------------
#Se emplea la misma variable imagen anterior

ndvi_opcion2 =imagen.expression(
      '(NIR - RED) / (NIR + RED)', {
      'NIR': imagen.select('B8'), 
      'RED': imagen.select('B4')})

El uso de expresiones para el cálculo de índices es muy recomendable cuando la ecuación no es una operación sencilla. 

Por ejemplo, supóngase que un proyecto es necesario calcular el índice EVI cuya fórmula es:

$EVI=2.5\cdot \frac{NIR-RED}{NIR+6\cdot RED-7.5\cdot BLUE+1}$

In [14]:
# Calculo de EVI mediante una expresión regular.
EVI = imagen.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': imagen.select('B8'), 
      'RED': imagen.select('B4'),
      'BLUE': imagen.select('B2')})


#### **Ejemplo de normalizedDifference**

Una alternativa más sencilla y limpia a la hora de escribir el código en este caso es mediante el uso del método ***normalizedDifference()***, pasando como parámetros las bandas con las que se quiere calcular la diferencia normalizada.

In [15]:
#Opción 3: Calculo de NDVI mediante normalizedDifference
#----------------------------------------------
#Se emplea la misma variable imagen anterior
ndvi_opcion3 = imagen.normalizedDifference(['B8', 'B4'])

#### **Representación de resultados**
Una vez que se ha calculado el NDVI de la escena a través de distintos métodos se va represetar el resultado en un visor interactivo.

In [16]:
#Llamada a la función centro para obtener las coordenada centrales de la escena
coordenadas_centrales=centro(imagen)
longitud=coordenadas_centrales[0]
latitud=coordenadas_centrales[1]
                 
# Creacion de un objeto de tipo folium map

mi_mapa = folium.Map(location=[latitud,longitud], zoom_start=12, height=1000)

# Añade la imagen al objeto mapa.
mi_mapa.add_ee_layer(imagen,vis_params_falso_color, 'Sentinel')
mi_mapa.add_ee_layer(ndvi_opcion1,{}, 'NDVI')

# Añade un panel de control de capas en el mapa.
mi_mapa.add_child(folium.LayerControl())

# Muestra el mapa.
display(mi_mapa)

La representación del NDVI anterior no facilita su interpretación. Por ello, en el siguiente ejemplo se va a crear una paleta de colores para representar la imágen de NDVI a través de la variable `ndvi_params`. En esta variable se ha establecido los valores máximos y minimos como 1 y -1 y la gama de colores a emplear en este rango a través del código hexadecimal de cada uno de los colores.

En este [link](https://htmlcolorcodes.com/es/) se puede consultar el código hexadecimal de un color así como sus valores en el espacio de color RGB, CMYK y HSL.

In [17]:
#Debe haberse ejecutado anteriormente el calculo del índice NDVI y todas las funciones creadas anteriormene

#Parámetros de visualización NDVI, incluye una paleta de color
ndvi_params = {'min':0, 'max': 1, 
          'palette': ['FFFFFF','CE7E45','DF923D','F1B555','FCD163','99B718',
          '74A901','66A000','529400','3E8601','207401','056201','004C00',
          '023B01','012E01','011D01','011301']
}

#Llamada a la función centro para obtener las coordenada centrales de la escena
coordenadas_centrales=centro(imagen)
longitud=coordenadas_centrales[0]
latitud=coordenadas_centrales[1]
                 
# Creacion de un objeto de tipo folium map

mi_mapa = folium.Map(location=[latitud,longitud], zoom_start=12, height=1000)

# Añade la imagen al objeto mapa.
mi_mapa.add_ee_layer(imagen,vis_params_falso_color, 'Sentinel')
mi_mapa.add_ee_layer(ndvi_opcion1,ndvi_params, 'NDVI opción 1')
mi_mapa.add_ee_layer(ndvi_opcion1,ndvi_params, 'NDVI opción 2')
mi_mapa.add_ee_layer(ndvi_opcion1,ndvi_params, 'NDVI opción 3')

# Añade un panel de control de capas en el mapa.
mi_mapa.add_child(folium.LayerControl())

# Muestra el mapa.
display(mi_mapa)

Visor representando la escena Sentinel así como los índices NDVI y EVI

In [18]:
#Presenciación de resultados para EVI
EVI_viz = {'min': 0, 'max': 1, 'palette': ['FF0000', '00FF00']} #Parámetros de visualización

vis_params_RGB = {
  'min': 0,
  'max': 3500,
  'bands': ['B4', 'B2', 'B2']}
  
# Creacion de un objeto de tipo folium map
# Se estan empleando los mismos valores centrales de la imagen que en codigo anteriores.
mi_mapa = folium.Map(location=[latitud,longitud], zoom_start=12, height=1000)

# Añade la imagen al objeto mapa.
mi_mapa.add_ee_layer(imagen,vis_params_RGB, 'Sentinel')
mi_mapa.add_ee_layer(ndvi_opcion1,ndvi_params,'NDVI')
mi_mapa.add_ee_layer(EVI,EVI_viz, 'EVI')

# Añade un panel de control de capas en el mapa.
mi_mapa.add_child(folium.LayerControl())

# Muestra el mapa.
display(mi_mapa)


Nota Python: La división de dos número enteros da como resultado un número entero. Por ejemplo 5 / 2 = 2. Si queremos obtener un valor decimal sera necesario multiplicar uno de los operadores por 1.0, es decir, 5*1.0/2=2.5.



En el siguiente script se trabaja con 3 escenas Sentinel correspondientes a los meses de abril, mayo y junio. Para cada una de las escenas se ha generado una nueva imagen con el resultado del índice NDVI. Además, cada una de las imagenes de NDVI ha pasado a formar parte de una banda de una variable imagen denominada `composiscion_ndvi`.

In [ ]:
imagen_abril=ee.Image('COPERNICUS/S2/20170412T110621_20170412T111708_T30SUG');
imagen_mayo=ee.Image('COPERNICUS/S2/20170502T110621_20170502T110937_T30SUG');
imagen_junio=ee.Image('COPERNICUS/S2/20170611T110621_20170611T111012_T30SUG');

vizParamsNIR = {
  'bands': ['B8','B3','B2'],
  'min': 1000, 
  'max': 5000,
  'gamma': 2
};

vizParams_NDVI = {
  'min': 0, 
  'max': 1,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301']
};

NDVI_abril= imagen_abril.normalizedDifference(['B8','B4']).rename('NDVI');
NDVI_mayo= imagen_mayo.normalizedDifference(['B8','B4']).rename('NDVI');
NDVI_junio= imagen_junio.normalizedDifference(['B8','B4']).rename('NDVI');

composicion_NDVI= NDVI_abril;

composicion_NDVI=composicion_NDVI.addBands(NDVI_mayo);
composicion_NDVI=composicion_NDVI.addBands(NDVI_junio);

vizNDVI_composicion = {
  'bands': ['NDVI','NDVI_1','NDVI_2'],
  'min': 0, 
  'max': 1,
  'gamma': 2
};

coordenadas_centrales=centro(imagen_abril)
longitud=coordenadas_centrales[0]
latitud=coordenadas_centrales[1]
mi_mapa = folium.Map(location=[latitud,longitud], zoom_start=11, height=1000)

# Añade la imagen al objeto mapa.
mi_mapa.add_ee_layer(imagen_abril,vizParamsNIR, 'Sentinel 2 - Abril')
mi_mapa.add_ee_layer(imagen_mayo,vizParamsNIR, 'Sentinel 2 - Mayo')
mi_mapa.add_ee_layer(imagen_junio,vizParamsNIR, 'Sentinel 2 - Junio')

mi_mapa.add_ee_layer(NDVI_abril,vizParams_NDVI, 'NDVI - Abril')
mi_mapa.add_ee_layer(NDVI_mayo,vizParams_NDVI, 'NDVI - Mayo')
mi_mapa.add_ee_layer(NDVI_junio,vizParams_NDVI, 'NDVI - Junio')

mi_mapa.add_ee_layer(composicion_NDVI,vizNDVI_composicion, 'Composicion NDVI')

# Añade un panel de control de capas en el mapa.
mi_mapa.add_child(folium.LayerControl())

# Muestra el mapa.
display(mi_mapa)


### 4.2 Operaciones booleanas, relacionales y condicionales.
Al igual que es posible aplicar operaciones algebraicas entre bandas es posible emplear condiciones para hacer una selección de pixeles que posteriormente se pueden emplear por ejemplo para generar una mascara sobre una imagen, para ello empleareos el método **updateMask()**.

En este ejemplo, aprovechando el resultado de celdas anteriores se representa, considerando la escena Sentinel 2 del mes de abril, aquellos píxeles de NDVI con valor superior o igual a 0.4, empleando para ello **gte()** (*greater than equal*).

In [ ]:
vegetacion=NDVI_abril.updateMask(NDVI_abril.gte(0.4));

mi_mapa = folium.Map(location=[latitud,longitud], zoom_start=11, height=1000)
mi_mapa.add_ee_layer(vegetacion,vizParams_NDVI, 'NDVI >= 0.4')

# Añade un panel de control de capas en el mapa.
mi_mapa.add_child(folium.LayerControl())

# Muestra el mapa.
display(mi_mapa)